In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import astropy
from astropy.io import fits
from PyAstronomy import pyasl
import hpfspec2
import glob
import random
import datetime
import ipywidgets as widget
import ipympl
import os
import pandas as pd
%matplotlib widget

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# RAW DATA

In [54]:
#where do we pull NPY files and put files to plot #etalon data1 or data2 and also LFC

#where we store the csv files basepath: /data4/HPF/etalon/data/DataToPlot/
storageSpace = ['DataToPlot_Etalon','DataToPlot_LFC','DataToPlot_Etalon_c2_sQ','DataToPlot_LFC_c2_sQ']
#where we pull the npy files basepath: /data3/HPF/etalon/data/Fitted_Data/
pullDataFrom = ['Fitted_Data_Etalon','Fitted_Data_LFC','Fitted_Data_Etalon_c2_sQ','Fitted_Data_LFC_c2_sQ']

#pull the NPY Files
allNPYfiles = sorted(glob.glob('/data3/HPF/etalon/data/Fitted_Data/'+pullDataFrom[2]+'/*.npy'))
print(len(allNPYfiles))
print(len(storageSpace))
print(len(pullDataFrom))

3443
4
4


In [60]:
#this cell loads all the files 
#and gets the centroid, FWHM, and velocities of multiple mode per file

for dataIndex in range(len(pullDataFrom)):  #we are pulling for each type of data
    allNPYfiles = sorted(glob.glob('/data3/HPF/etalon/data/Fitted_Data/'+pullDataFrom[dataIndex]+'/*.npy')) #pull npy files
    
    #SETTINGS BEGIN
    
    allData = True #if we want to cut off the data ends
    fileStart = 0
    fileCutOff = 5000
    storeInOther = True #use this if doing small test or data (ONLY HAVE TURNED OFF IF DOING THE LARGE SCALE CHANNELS)

    lookForBAD = False #look for bad files

    Slim_Analysis = False #if we want a quick analysis (NOT EVERY FILE)
    slimness = 10 #larger means less data, this skips to every nth file

    splitByPixel = False
    pixelToStartAt = 1537
    pixelToEndBy = 1792

    #if we want to look at a certain number of orders or modes
    num_of_orders = 1 
    num_of_modes = 20 
    all_orders = True #do we want all orders or modes
    all_modes = True
    start_order = 0 #if only looking at certain orders or modes, where do you wanna look around
    start_mode = 60
    
    AVG_over_time = False #do we want to average the data from multiple nearby files to get rid of noise? #NOT IMPLEMENTED
    files_to_AVG_over = 10 #if AVG_over_time

    #what average method do we want
    wantAVG = False
    wantMED = False
    wantBIWEIGHT = True
    
    #SETTINGS END
    
    #CODE BEGINS (DONT CHANGE ANYTHING BELOW)

    #initialize data structures
    centroidsWL = []
    fwhmWL = []
    centroidsPIX = []
    fwhmPIX = []
    v_list = []
    times = []
    if lookForBAD:
        cur_bad_files = file1.read().splitlines() #pull the current bad files if looking for weird data #replace file1 with file that has bad file path names
        bad_files = []


    if not Slim_Analysis:
        slimness = 1
    if allData:
        fileStart = 0
        fileCutOff = len(allNPYfiles)

    rangeToLook = range(fileStart,fileCutOff,slimness)

    for i in rangeToLook: #going over every file
        fitdata = np.load(allNPYfiles[i],allow_pickle=True)[()]
        new_centroid_WL = []
        new_fwhm_WL = []
        new_centroid_PIX = []
        new_fwhm_PIX = []
        temp_v_list = []
        speed_light = 2.99792458*10**8    

        if i==fileStart: #pull the original spectrum to calculate v-shifts off of
            original_spectrum = fitdata


        if all_orders: #adding all orders to get the average over all the orders in a spectrum
            num_of_orders = len(fitdata.keys())
            start_order = 0



        for j in range(num_of_orders): #going over every order
            if all_modes or splitByPixel: #adding all the modes of an order to get the average over that order
                num_of_modes = len(fitdata[j+start_order].keys())
                start_mode = 0
                mode_count = 0

            for k in range(num_of_modes): #going over every mode and pulling everything we need from that mode
                
                #isolate the current mode
                curMode = fitdata[j+start_order][k+start_mode]
                curPix = curMode['centroid_pix']
                
                #first test if we are too far
                if (curPix>pixelToEndBy) and splitByPixel:
                    break #if we have gone past our desired pixel, we want to stop
                #pull all information
                if ((curPix>=pixelToStartAt) and splitByPixel) or not splitByPixel:
                    cur_centroid_WL = curMode['centroid_wl']
                    cur_fwhm_WL = curMode['fwhm_wl']
                    cur_centroid_PIX = curMode['centroid_pix']
                    cur_fwhm_PIX = curMode['fwhm_pix']

                    new_centroid_WL.append(cur_centroid_WL)
                    new_fwhm_WL.append(cur_fwhm_WL)
                    new_centroid_PIX.append(cur_centroid_PIX)
                    new_fwhm_PIX.append(cur_fwhm_PIX)

                    #calculate and save the velocity values
                    orig_wl = (original_spectrum[j+start_order][k+start_mode]['centroid_wl'])*(10**(-10))
                    cur_wl = (curMode['centroid_wl'])*(10**(-10))
                    cur_v = speed_light*(cur_wl-orig_wl)/orig_wl 
                    temp_v_list.append(cur_v)

                    #keep track of where we are looking to use for naming the file
                    if mode_count==0:
                        mode_location = k
                    mode_count+=1

                if lookForBAD: #pull any bad files
                    #test the fwhm wl, if we need we can do it for centroids too, but fwhm is more useful
                    if np.isnan(cur_fwhm_WL) or (int(cur_fwhm_WL) not in range(0,2)):
                        if allNPYfiles[i] not in cur_bad_files and allfiles[i] not in bad_files:
                            bad_files.append(allfiles[i])

        centroidsWL.append(new_centroid_WL)
        fwhmWL.append(new_fwhm_WL)
        centroidsPIX.append(new_centroid_PIX)
        fwhmPIX.append(new_fwhm_PIX)
        v_list.append(temp_v_list)

        if pullDataFrom[dataIndex] == 'Fitted_Data_LFC':
            curTime = allNPYfiles[i][57:72]
        elif pullDataFrom[dataIndex] == 'Fitted_Data_Etalon_c2_sQ':
            curTime = allNPYfiles[i][66:81]
        elif pullDataFrom[dataIndex] == 'Fitted_Data_LFC_c2_sQ':
                curTime = allNPYfiles[i][63:78]
        else:
            curTime = allNPYfiles[i][60:75]
        times.append(datetime.datetime.strptime(curTime,'%Y%m%dT%H%M%S'))


    #AVERAGE
    
    average_array_centroid_wl = []
    average_array_centroid_pix = []
    average_array_fwhm_wl = []
    average_array_fwhm_pix = []
    average_array_v = []
    
    for eachArray in centroidsWL:
        if wantAVG:
            average_centroid = np.nanmean(eachArray)
        elif wantMED:
            average_centroid = np.nanmedian(eachArray)
        else:
            average_centroid = astropy.stats.biweight_location(eachArray,ignore_nan=True)

        average_array_centroid_wl.append(average_centroid)

    for eachArray in centroidsPIX:
        if wantAVG:
            average_centroid = np.nanmean(eachArray)
        elif wantMED:
            average_centroid = np.nanmedian(eachArray)
        else:
            average_centroid = astropy.stats.biweight_location(eachArray,ignore_nan=True)

        average_array_centroid_pix.append(average_centroid)

    for eachArray in fwhmWL:
        if wantAVG:
            average_fwhm = np.nanmean(eachArray)
        elif wantMED:
            average_fwhm = np.nanmedian(eachArray)
        else:
            average_fwhm = astropy.stats.biweight_location(eachArray,ignore_nan=True)

        average_array_fwhm_wl.append(average_fwhm)

    for eachArray in fwhmPIX:
        if wantAVG:
            average_fwhm = np.nanmean(eachArray)
        elif wantMED:
            average_fwhm = np.nanmedian(eachArray)
        else:
            average_fwhm = astropy.stats.biweight_location(eachArray,ignore_nan=True)

        average_array_fwhm_pix.append(average_fwhm)

    for eachArray in v_list:
        if wantAVG:
            average_v = np.nanmean(eachArray)
        elif wantMED:
            average_v = np.nanmedian(eachArray)
        else:
            average_v = astropy.stats.biweight_location(eachArray,ignore_nan=True)

        average_array_v.append(average_v)

    #SAVING THE DATA
       
    #naming structure
    if wantMED:
        AVG = 'Median '
    elif wantBIWEIGHT:
        AVG = 'Bi-Weight '
    else:
        AVG = 'Average '

    if Slim_Analysis:
        part1 = 'Slim'
    else:
        part1 = 'Full'
    if all_orders:
        part2 = 'All_Orders'
    else:
        part2 = str(num_of_orders)+'OrdersAt'+str(start_order)
    if all_modes:
        part3 = 'All_Modes'
    else:
        part3 = str(mode_count)+'ModesAt'+str(mode_location)

    name = part1+'RawData_'+part2+'_'+part3


    # intialise data of lists for RAW data.
    data = {'Date':times,AVG+'Centroid (Å)':average_array_centroid_wl,AVG+'FWHM (Å)':average_array_fwhm_wl,AVG+'Centroid (pixels)':average_array_centroid_pix,AVG+'FWHM (pixels)':average_array_fwhm_pix,AVG+'Velocity Shifts (m/s)':average_array_v}

    # Create DataFrame
    df_out = pd.DataFrame(data)
    
    # data to csv
    if storeInOther:
        namingScheme = '/Other/'
    else:
        namingScheme = '/'
    df_out.to_csv('/data4/HPF/etalon/data/DataToPlot/'+storageSpace[dataIndex]+namingScheme+name+'.csv')

In [8]:
print(bad_files) #if it is pulling bad files, print them out

NameError: name 'bad_files' is not defined